In [1]:
import numpy as np
import pandas as pd
import os

os.chdir("../")
pd.set_option("display.max_columns", None)

In [2]:
year = 2017
election_round = "first_round" # european_parliament first_round

In [3]:
# Read full data
if year == 2022:
    df = pd.read_excel(f"data/France/{year}_{election_round}.xlsx")
    df["polling_id"] = df["Code du département"].astype(str) + "-" + df["Code de la commune"].astype(str) + "-" + df["Code du b.vote"].astype(str)
    
elif year == 2017:
    df = pd.read_csv(f"data/France/{year}_{election_round}.txt", delimiter=";", encoding="latin-1")
    df = df.reset_index()
    df.columns = [f"level_{x}" for x in range(len(list(df)))]
    
    df["polling_id"] = df["level_0"].astype(str) + "-" + df["level_4"].astype(str) + "-" +\
        "-" + df["level_22"].astype(str)
    
    if election_round == "runoff":
        candidates = {
            "level_25": "Emmanuel MACRON",
            "level_32": "Marine LE PEN"
        }
        
    else:
        candidates = {
            "level_25": "Nicolas DUPONT-AIGNAN",
            "level_32": "Marine LE PEN",
            "level_39": "Emmanuel MACRON",
            "level_46": "Benoît HAMON",
            "level_53": "Nathalie ARTHAUD",
            "level_60": "Philippe POUTOU",
            "level_67": "Jacques CHEMINADE",
            "level_74": "Jean LASSALLE",
            "level_81": "Jean-Luc MÉLENCHON",
            "level_88": "François ASSELINEAU",
            "level_95": "François FILLON"
        }
        
    df = df.rename(columns=candidates)
    df = df.rename(columns={
        "level_8": "Abstentions",
        "level_12": "Blancs",
        "level_15": "Nuls"})
    df_filtered = df[["polling_id"] + list(candidates.values())]
    
elif year == 2012:
    df = pd.read_csv(f"data/France/{year}.txt", delimiter=";", encoding="latin-1")
    df = pd.DataFrame(np.vstack([df.columns, df]))
    rnd = 1 if election_round == "first_round" else 2
    df = df.reset_index()
    df.columns = [f"level_{x}" for x in range(len(list(df)))]
    df = df[df["level_1"] == rnd].copy()

    df["polling_id"] = df["level_2"].astype(str) + "-" + df["level_3"].astype(str)\
        + "-" + df["level_6"].astype(str) + "-" + df["level_7"].astype(str)

    df["Candidate"] = df["level_13"] + " " + df["level_12"]
    df = df.rename(columns={"level_15": "Votes"})

elif year in [2002, 2007]:
    df = pd.read_csv(f"data/France/{year}.txt", delimiter=";", header=16, encoding="latin-1")
    rnd = 1 if election_round == "first_round" else 2
    df = df.reset_index()
    df.columns = [f"level_{x}" for x in range(len(list(df)))]
    df = df[df["level_0"] == rnd].copy()
    df["level_1"] = df["level_1"].astype(str).str.zfill(2)

    df["polling_id"] = df["level_1"].astype(str) + "-" + df["level_2"].astype(str)\
        + "-" + df["level_4"].astype(str)

    df["Candidate"] = df["level_10"] + " " + df["level_9"]
    df = df.rename(columns={"level_12": "Votes"})

df.head()

C:\Users\cnava\AppData\Local\Temp\ipykernel_12580\1195840770.py:7: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/France/{year}_{election_round}.txt", delimiter=";", encoding="latin-1")


,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,Abstentions,level_9,level_10,level_11,Blancs,level_13,level_14,Nuls,level_16,level_17,level_18,level_19,level_20,level_21,level_22,level_23,level_24,Nicolas DUPONT-AIGNAN,level_26,level_27,level_28,level_29,level_30,level_31,Marine LE PEN,level_33,level_34,level_35,level_36,level_37,level_38,Emmanuel MACRON,level_40,level_41,level_42,level_43,level_44,level_45,Benoît HAMON,level_47,level_48,level_49,level_50,level_51,level_52,Nathalie ARTHAUD,level_54,level_55,level_56,level_57,level_58,level_59,Philippe POUTOU,level_61,level_62,level_63,level_64,level_65,level_66,Jacques CHEMINADE,level_68,level_69,level_70,level_71,level_72,level_73,Jean LASSALLE,level_75,level_76,level_77,level_78,level_79,level_80,Jean-Luc MÉLENCHON,level_82,level_83,level_84,level_85,level_86,level_87,François ASSELINEAU,level_89,level_90,level_91,level_92,level_93,level_94,François FILLON,level_96,level_97,polling_id
0,1,Ain,4,4ème circonscription,1,L'Abergement-Clémenciat,1.0,598,92,"15,38",506,"84,62",2,"0,33","0,40",9,"1,51","1,78",495,"82,78","97,83",1,M,DUPONT-AIGNAN,Nicolas,34,"5,69","6,87",2,F,LE PEN,Marine,126,"21,07","25,45",3,M,MACRON,Emmanuel,119,"19,90","24,04",4,M,HAMON,Benoît,29,"4,85","5,86",5,F,ARTHAUD,Nathalie,4,"0,67","0,81",6,M,POUTOU,Philippe,4,"0,67","0,81",7,M,CHEMINADE,Jacques,2,"0,33","0,40",8,M,LASSALLE,Jean,2,"0,33","0,40",9,M,MÉLENCHON,Jean-Luc,59,"9,87","11,92",10,M,ASSELINEAU,François,6,"1,00","1,21",11,M,FILLON,François,110,"18,39","22,22",1-1--M
1,1,Ain,5,5ème circonscription,2,L'Abergement-de-Varey,1.0,209,25,"11,96",184,"88,04",6,"2,87","3,26",2,"0,96","1,09",176,"84,21","95,65",1,M,DUPONT-AIGNAN,Nicolas,6,"2,87","3,41",2,F,LE PEN,Marine,48,"22,97","27,27",3,M,MACRON,Emmanuel,37,"17,70","21,02",4,M,HAMON,Benoît,13,"6,22","7,39",5,F,ARTHAUD,Nathalie,2,"0,96","1,14",6,M,POUTOU,Philippe,2,"0,96","1,14",7,M,CHEMINADE,Jacques,0,"0,00","0,00",8,M,LASSALLE,Jean,0,"0,00","0,00",9,M,MÉLENCHON,Jean-Luc,33,"15,79","18,75",10,M,ASSELINEAU,François,1,"0,48","0,57",11,M,FILLON,François,34,"16,27","19,32",1-2--M
2,1,Ain,5,5ème circonscription,4,Ambérieu-en-Bugey,1.0,1116,233,"20,88",883,"79,12",17,"1,52","1,93",6,"0,54","0,68",860,"77,06","97,40",1,M,DUPONT-AIGNAN,Nicolas,58,"5,20","6,74",2,F,LE PEN,Marine,233,"20,88","27,09",3,M,MACRON,Emmanuel,156,"13,98","18,14",4,M,HAMON,Benoît,47,"4,21","5,47",5,F,ARTHAUD,Nathalie,6,"0,54","0,70",6,M,POUTOU,Philippe,15,"1,34","1,74",7,M,CHEMINADE,Jacques,0,"0,00","0,00",8,M,LASSALLE,Jean,6,"0,54","0,70",9,M,MÉLENCHON,Jean-Luc,216,"19,35","25,12",10,M,ASSELINEAU,François,7,"0,63","0,81",11,M,FILLON,François,116,"10,39","13,49",1-4--M
3,1,Ain,5,5ème circonscription,4,Ambérieu-en-Bugey,2.0,1128,256,"22,70",872,"77,30",19,"1,68","2,18",3,"0,27","0,34",850,"75,35","97,48",1,M,DUPONT-AIGNAN,Nicolas,42,"3,72","4,94",2,F,LE PEN,Marine,234,"20,74","27,53",3,M,MACRON,Emmanuel,183,"16,22","21,53",4,M,HAMON,Benoît,51,"4,52","6,00",5,F,ARTHAUD,Nathalie,5,"0,44","0,59",6,M,POUTOU,Philippe,16,"1,42","1,88",7,M,CHEMINADE,Jacques,1,"0,09","0,12",8,M,LASSALLE,Jean,8,"0,71","0,94",9,M,MÉLENCHON,Jean-Luc,175,"15,51","20,59",10,M,ASSELINEAU,François,11,"0,98","1,29",11,M,FILLON,François,124,"10,99","14,59",1-4--M
4,1,Ain,5,5ème circonscription,4,Ambérieu-en-Bugey,3.0,1116,227,"20,34",889,"79,66",11,"0,99","1,24",4,"0,36","0,45",874,"78,32","98,31",1,M,DUPONT-AIGNAN,Nicolas,47,"4,21","5,38",2,F,LE PEN,Marine,218,"19,53","24,94",3,M,MACRON,Emmanuel,191,"17,11","21,85",4,M,HAMON,Benoît,53,"4,75","6,06",5,F,ARTHAUD,Nathalie,4,"0,36","0,46",6,M,POUTOU,Philippe,12,"1,08","1,37",7,M,CHEMINADE,Jacques,0,"0,00","0,00",8,M,LASSALLE,Jean,11,"0,99","1,26",9,M,MÉLENCHON,Jean-Luc,182,"16,31","20,82",10,M,ASSELINEAU,François,7,"0,63","0,80",11,M,FILLON,François,149,"13,35","17,05",1-4--M


In [4]:
df_spoilt = pd.DataFrame()

if year in [2017, 2022]:

    df_spoilt = df[["polling_id", "Abstentions", "Blancs", "Nuls"]]\
        .rename(columns={"Abstentions": "ABSTENTION", "Blancs": "BLANK", "Nuls": "NULL"})\
        .melt(id_vars=["polling_id"], var_name="candidate")

    df_spoilt["flag_candidates"] = 0

df_spoilt.head()

,polling_id,candidate,value,flag_candidates
0,1-1--M,ABSTENTION,92,0
1,1-2--M,ABSTENTION,25,0
2,1-4--M,ABSTENTION,233,0
3,1-4--M,ABSTENTION,256,0
4,1-4--M,ABSTENTION,227,0


In [5]:
if year == 2022:

    if election_round == "runoff":
        df = df.rename(columns={
            "Voix": "Emmanuel MACRON",
            "Unnamed: 32": "Marine LE PEN"
        })

        df_filtered = df[[
            "polling_id",
    #         "Code du département", 
    #         "Libellé de la commune", 
    #         "Code du b.vote", 
            "Emmanuel MACRON",
            "Marine LE PEN", 
        ]]
    else:
        df = df.rename(columns={
            "Voix": "Nathalie ARTHAUD",
            "Unnamed: 32": "Fabien ROUSSEL",
            "Unnamed: 39": "Emmanuel MACRON",
            "Unnamed: 46": "Jean LASSALLE",
            "Unnamed: 53": "Marine LE PEN",
            "Unnamed: 60": "Éric ZEMMOUR",
            "Unnamed: 67": "Jean-Luc MÉLENCHON",
            "Unnamed: 74": "Anne HIDALGO",
            "Unnamed: 81": "Yannick JADOT",
            "Unnamed: 88": "Valérie PÉCRESSE",
            "Unnamed: 95": "Philippe POUTOU",
            "Unnamed: 102": "Nicolas DUPONT-AIGNAN"
        })

        df_filtered = df[[
            "polling_id",
    #         "Code du département", 
    #         "Libellé de la commune", 
    #         "Code du b.vote", 
            "Nathalie ARTHAUD", 
            "Fabien ROUSSEL", 
            "Emmanuel MACRON",
            "Jean LASSALLE", 
            "Marine LE PEN", 
            "Éric ZEMMOUR", 
            "Jean-Luc MÉLENCHON", 
            "Anne HIDALGO",
            "Yannick JADOT",
            "Valérie PÉCRESSE",
            "Philippe POUTOU",
            "Nicolas DUPONT-AIGNAN"
        ]]

In [6]:
if year in [2017, 2022]:
    df_filtered = df_filtered.melt(
        id_vars=["polling_id"], 
        var_name="Candidate", 
        value_name="Votes"
    )

elif year in [2002, 2007, 2012]:
    df_filtered = df[["polling_id", "Candidate", "Votes"]].copy()
# df_filtered["polling_id"] = df_filtered["Libellé de la commune"].astype(str) + "-" + df_filtered["Code du b.vote"].astype(str)

df_filtered.head()

,polling_id,Candidate,Votes
0,1-1--M,Nicolas DUPONT-AIGNAN,34
1,1-2--M,Nicolas DUPONT-AIGNAN,6
2,1-4--M,Nicolas DUPONT-AIGNAN,58
3,1-4--M,Nicolas DUPONT-AIGNAN,42
4,1-4--M,Nicolas DUPONT-AIGNAN,47


In [7]:
## Check whether there are polling ids with zero votes
zeros = df_filtered.groupby("polling_id").agg({"Votes": "sum"})
zeros = list(zeros[zeros["Votes"] == 0].index)

df_filtered = df_filtered[~df_filtered["polling_id"].isin(zeros)].groupby(["polling_id", "Candidate"]).agg({"Votes": "sum"})        
df_filtered["rate"] = df_filtered.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
df_filtered = df_filtered.reset_index()
df_filtered = df_filtered.rename(columns={"Votes": "value", "Candidate": "candidate"})
df_filtered = df_filtered[["polling_id", "candidate", "rate", "value"]]
df_filtered["rank"] = df_filtered.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)
df_filtered["flag_candidates"] = 1

In [8]:
data = pd.concat([
    df_filtered,
    df_spoilt
])

data.head()

,polling_id,candidate,rate,value,rank,flag_candidates
0,1-1--M,Benoît HAMON,0.058586,29,6.0,1
1,1-1--M,Emmanuel MACRON,0.240404,119,2.0,1
2,1-1--M,François ASSELINEAU,0.012121,6,7.0,1
3,1-1--M,François FILLON,0.222222,110,3.0,1
4,1-1--M,Jacques CHEMINADE,0.004040,2,10.0,1


In [9]:
data.to_csv(f"data_output/France/{year}_{election_round}.csv.gz", compression="gzip", index=False)

In [10]:
data.candidate.unique()

array(['Benoît HAMON', 'Emmanuel MACRON', 'François ASSELINEAU',
       'François FILLON', 'Jacques CHEMINADE', 'Jean LASSALLE',
       'Jean-Luc MÉLENCHON', 'Marine LE PEN', 'Nathalie ARTHAUD',
       'Nicolas DUPONT-AIGNAN', 'Philippe POUTOU', 'ABSTENTION', 'BLANK',
       'NULL'], dtype=object)

In [11]:
# https://ec.europa.eu/eurostat/documents/345175/629341/NUTS2021.xlsx

In [12]:
df_nuts = pd.read_excel(
    "https://ec.europa.eu/eurostat/documents/345175/629341/NUTS2021.xlsx",
    sheet_name="NUTS & SR 2021"
)
df_nuts["country_code"] = df_nuts["Code 2021"].str[0:2]

df_nuts = df_nuts[df_nuts["country_code"] == "FR"]#.fillna(method="ffill")

for col in ["NUTS level 1", "NUTS level 2"]:
    df_nuts[col] = df_nuts[col].fillna(method="ffill")
    
df_nuts.head()

def get_nuts_level(df, level=1):
    output_df = df[["Code 2021", f"NUTS level {level}", "NUTS level"]].dropna()
    output_df = output_df[output_df["NUTS level"] == level]
    output_df = output_df[["Code 2021", f"NUTS level {level}"]]
    output_df = output_df.rename(columns={"Code 2021": f"nuts_{level}", f"NUTS level {level}": f"nuts_{level}_name"})
    output_df = output_df.reset_index(drop=True)

    return output_df

df_nuts_1 = get_nuts_level(df_nuts, level=1)
df_nuts_2 = get_nuts_level(df_nuts, level=2)

df_nuts_3 = df_nuts[["Code 2021", "NUTS level 1", "NUTS level 2", "NUTS level 3"]].dropna()
df_nuts_3 = df_nuts_3.rename(
    columns={
        "Code 2021": "nuts_3", 
        "NUTS level 3": "department",
        "NUTS level 1": "nuts_1_name",
        "NUTS level 2": "nuts_2_name"
    })

df_nuts_3 = pd.merge(df_nuts_3, df_nuts_2, on="nuts_2_name")
df_nuts_3 = pd.merge(df_nuts_3, df_nuts_1, on="nuts_1_name")
df_nuts_3["department"] = df_nuts_3["department"].str.strip().str.replace("’", "'")

df_nuts_3.head()

,nuts_3,nuts_1_name,nuts_2_name,department,nuts_2,nuts_1
0,FR101,Ile-de-France,Ile-de-France,Paris,FR10,FR1
1,FR102,Ile-de-France,Ile-de-France,Seine-et-Marne,FR10,FR1
2,FR103,Ile-de-France,Ile-de-France,Yvelines,FR10,FR1
3,FR104,Ile-de-France,Ile-de-France,Essonne,FR10,FR1
4,FR105,Ile-de-France,Ile-de-France,Hauts-de-Seine,FR10,FR1


In [13]:
df_nuts_3.head()

,nuts_3,nuts_1_name,nuts_2_name,department,nuts_2,nuts_1
0,FR101,Ile-de-France,Ile-de-France,Paris,FR10,FR1
1,FR102,Ile-de-France,Ile-de-France,Seine-et-Marne,FR10,FR1
2,FR103,Ile-de-France,Ile-de-France,Yvelines,FR10,FR1
3,FR104,Ile-de-France,Ile-de-France,Essonne,FR10,FR1
4,FR105,Ile-de-France,Ile-de-France,Hauts-de-Seine,FR10,FR1


In [14]:
df_deps = pd.read_csv("https://raw.githubusercontent.com/alan-eu/french-departments/main/departments.csv")
df_deps = df_deps.rename(columns={"code": "department_id", "name": "department"})
df_deps = df_deps.replace({"Eure-et-Loire": "Eure-et-Loir"})
df_deps["department_id"] = df_deps["department_id"].astype(str).str.zfill(2)
df_deps.head()

,department_id,department,nameWithPreposition
0,01,Ain,dans l’Ain
1,02,Aisne,dans l’Aisne
2,03,Allier,dans l’Allier
3,04,Alpes-de-Haute-Provence,dans les Alpes-de-Haute-Provence
4,05,Hautes-Alpes,dans les Hautes-Alpes


In [15]:
if year in [2007, 2012]:
    df_nuts_3 = pd.merge(df_deps, df_nuts_3, on="department")

In [16]:
set(df_deps.department) - set(df_nuts_3.department)

set()

In [19]:
if year == 2017:

    df_location = df[["polling_id", "level_0", "level_1", "level_2", "level_3", "level_4", "level_5", "level_22"]]

    df_location = df_location.rename(columns={
        "level_0": "department_id",
        "level_1": "department",
        "level_2": "circunscription_id",
        "level_3": "circunscription",
        "level_4": "commune_id",
        "level_5": "commune",
        "level_22": "polling_place"
    })
    
elif year == 2022:
    df_location = df[["Code du département", "Libellé du département", "Code de la circonscription", "Libellé de la circonscription",
       "Code de la commune", "Libellé de la commune", "Code du b.vote", "Inscrits", "Abstentions", "polling_id"]]

    df_location = df_location.rename(columns={
        "Code du département": "department_id",
        "Libellé du département": "department",
        "Code de la circonscription": "circunscription_id",
        "Libellé de la circonscription": "circunscription",
        "Code de la commune": "commune_id",
        "Libellé de la commune": "commune",
        "Code du b.vote": "polling_place",
        "Inscrits": "electors",
        "Abstentions": "abstentions"
    })
    
elif year == 2012:
    df_location = df[["level_2", "level_3", "level_4", "level_8", "polling_id"]].copy().drop_duplicates()

    df_location = df_location.rename(columns={
        "level_2": "department_id",
        "level_3": "commune_id",
        "level_4": "commune",
        "level_8": "electors"
    })
    
elif year in [2002, 2007]:
    df_location = df[["level_1", "level_2", "level_3", "level_4", "level_5", "level_6", "polling_id"]].copy().drop_duplicates()

    df_location = df_location.rename(columns={
        "level_1": "department_id",
        "level_2": "commune_id",
        "level_3": "commune",
        "level_4": "polling_station",
        "level_5": "inscrits",
        "level_6": "voters"
    })

if year in [2017, 2022]:
    df_location = pd.merge(df_location, df_nuts_3, on="department", how="left")
    df_location["department_id"] = df_location["department_id"].astype(str).str.zfill(2)

elif year in [2007, 2012]:
    df_location["department_id"] = df_location["department_id"].astype(str).str.zfill(2)
    df_nuts_3["department_id"] = df_nuts_3["department_id"].astype(str).str.zfill(2)

    df_location = df_location[["polling_id", "department_id", "commune_id"]].drop_duplicates()
    df_location = pd.merge(df_location, df_nuts_3, on="department_id", how="left")#.dropna(subset=["department"])

df_location["commune_id"] = df_location["department_id"].astype(str).str.zfill(2) + df_location["commune_id"].astype(str).str.zfill(3)
df_location.to_csv(f"data_output/France/{year}_{election_round}_location.csv.gz", compression="gzip", index=False)

In [ ]:
# dd = pd.merge(df_filtered, df_location[["polling_id", "nuts_2"]].drop_duplicates(), how="inner", on="polling_id")
# dd.groupby("polling_id").count().sort_values("candidate")